In [2]:
from nba_api.stats.static import teams
import pandas as pd 


In [4]:
OKC_data  = teams.find_teams_by_full_name("Oklahoma City Thunder")
OKC_id = OKC_data[0]['id'] # 1610612760 team id 

PAC_data = teams.find_teams_by_full_name("Indiana Pacers")
PAC_id = PAC_data[0]['id'] # 1610612754 team id 

In [5]:
from nba_api.stats.endpoints import leaguegamefinder

finder = leaguegamefinder.LeagueGameFinder(
player_or_team_abbreviation= "T",
team_id_nullable=1610612760,
date_from_nullable="06/05/2025",
date_to_nullable="06/05/2025"
)

g1 = finder.get_data_frames()[0] 
g1_id= g1["GAME_ID"] #0042400401	


# Players Data

In [6]:
from nba_api.stats.endpoints import boxscoretraditionalv2
box = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=g1_id)
players = box.player_stats.get_data_frame()
players.to_csv("player_data.csv")

# Team Data

In [7]:
teams = box.team_stats.get_data_frame()
teams.to_csv("team_data.csv")

# Advanced player and team stats

In [8]:
from nba_api.stats.endpoints import boxscoreadvancedv2
adv_box = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=g1_id)
adv_player_df = adv_box.player_stats.get_data_frame() #advanced player stats

adv_team_df = adv_box.team_stats.get_data_frame()
adv_team_df.to_csv("adv_team.csv")#advanced team stats

In [9]:
adv_player_df.to_csv("adv_players.csv")

# Play by play data

In [10]:
from nba_api.stats.endpoints import playbyplayv2
pbp = playbyplayv2.PlayByPlayV2(game_id=g1_id)
pbp_df = pbp.get_data_frames()[0]
pbp_df # play by play stats 
pbp_df.to_csv("plays.csv")

# Hustle stats

In [ ]:
from nba_api.stats.endpoints import BoxScoreHustleV2

hustle = BoxScoreHustleV2(game_id=g1_id)
hustle_df = hustle.get_data_frames()[0]
hustle_df.to_csv("hustle.csv")

# Win Probability PBP

In [32]:
from nba_api.stats.endpoints import WinProbabilityPBP

w_pct_pbp = WinProbabilityPBP(game_id=g1_id,run_type="each second")
w_pct_pbp_df = w_pct_pbp.get_data_frames()[0]
w_pct_pbp_df.to_csv("win_prob.csv")

In [48]:
from nba_api.stats.endpoints import CommonTeamRoster, ShotChartDetail

team_id     = PAC_id        
season      = '2024-25'            # full season string
season_type = 'Playoffs'           # 'Regular Season' | 'Pre Season' | 'Playoffs' | 'All Star'
game_id     = g1_id        
# ────────────────────────────────────────────────────────────────────────────────

# 1. Fetch the team’s roster for the season
roster = CommonTeamRoster(team_id=team_id, season=season)
roster_df = roster.get_data_frames()[0]
player_ids = roster_df['PLAYER_ID'].tolist()

# 2. Loop through each player and pull their shot chart detail for that game
shots_list = []
for pid in player_ids:
    shot_detail = ShotChartDetail(
        context_measure_simple   = 'PTS',
        last_n_games             = 0,
        league_id                = '00',
        season_type_all_star     = season_type,
        season_nullable          = season,
        team_id                  = 0,
        player_id                = pid,
        game_id_nullable         = game_id,
        date_from_nullable       = '06/05/2025',
        date_to_nullable         = '06/05/2025',
        game_segment_nullable    = '',
        period                   = 0,
        outcome_nullable         = '',
        location_nullable        = '',
        month                    = 0,
        vs_conference_nullable   = '',
        vs_division_nullable     = '',
        opponent_team_id         = 0,
        rookie_year_nullable     = '',
        season_segment_nullable  = '',
        start_period_nullable    = '',
        end_period_nullable      = '',
        start_range_nullable     = '',
        end_range_nullable       = '',
        range_type_nullable      = '',
        point_diff_nullable      = '',
        position_nullable        = '',
        player_position_nullable = ''
    )
    # [0] = LeagueAverages, [1] = Shot_Chart_Detail
    _, df_shots = shot_detail.get_data_frames()
    if not df_shots.empty:
        shots_list.append(df_shots)
team_shot_detail_df = pd.concat(shots_list, ignore_index=True)

# Four Factors

In [37]:
from nba_api.stats.endpoints import boxscorefourfactorsv2
start_period = 1             # period to start (1 = Q1)
end_period   = 4             # period to end (4 = Q4; use 5 for OT if needed)
range_type   = 0             # 0 = entire period range
start_range  = 0             # seconds into period to start (0 = beginning)
end_range    = 0             # seconds into period to end (0 = full period)

four_factors = boxscorefourfactorsv2.BoxScoreFourFactorsV2(
    game_id    = g1_id,
    start_period = start_period,
    end_period   = end_period,
    range_type   = range_type,
    start_range  = start_range,
    end_range    = end_range
)


players_four_factors, teams_four_factors = four_factors.get_data_frames()

players_four_factors.to_csv("players_four_factors.csv")

In [38]:
teams_four_factors.to_csv("teams_four_factors.csv")